- Jose Pablo Ponce 19092
- Gabriel Quiroz

In [173]:
#Importar las librerías

import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from yellowbrick.features import Rank2D
from yellowbrick.features import RadViz

## Parte 1 – Ingeniería de características


## Exploración de datos


Revise la data y realice las operaciones necesarias para unificar los datasets y que el dataset final contenga el mensaje del correo y la etiqueta que indique si es SPAM o no.

In [174]:
df1 = pd.read_csv('./completeSpamAssassin.csv/completeSpamAssassin.csv')  
df2 = pd.read_csv('./enronSpamSubset.csv/enronSpamSubset.csv')  

In [175]:
pd.options.display.max_colwidth = 200

In [176]:
print(df1.sample(n=5).head())


      Unnamed: 0  \
2831        2831   
22            22   
3005        3005   
1520        1520   
552          552   

                                                                                                                                                                                                         Body  \
2831  Hardware acceleration for SSL makes sense since PKI can slow down a\ntransaction by as much as 1,000-fold. Per this article, XML formatting only\nincreases doc size by about 20-fold.. I'm not sure...   
22                                                                                                                                                                                                      empty   
3005  Hi,Just the kind of announce I make once in a while :\n- I've rebuilt a new "alsaplayer" package based on Angle's one.\n- I've rebuilt a hopefully final version "-fr8" of the "alsa-driver" and\n  ...   
1520  Long time no chat!How have you been? 

In [177]:
print(df2.sample(n=5).head())

      Unnamed: 0.1  Unnamed: 0  \
5069         29751       29751   
6490         33244       33244   
1241         12670       12670   
6581         18836       18836   
1710         10603       10603   

                                                                                                                                                                                                         Body  \
5069  Subject: start date : 1 / 5 / 02 ; hourahead hour : 17 ;\n start date : 1 / 5 / 02 ; hourahead hour : 17 ; no ancillary schedules awarded . no variances detected .\n log messages :\n parsing file ...   
6490  Subject: re : tw unsubscribed capacity\n i added an extra " printable page " button on the page so that the entire page can be printed out .\n michelle , please let me know if you still have probl...   
1241  Subject: site update mon , 27 jun 2005 .\n subject : site update mon , 27 jun 2005 .\n thank you for using our online store and for your previous order . we have 

In [178]:
df1 = df1.drop(columns=['Unnamed: 0'])
df2 = df2.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
print(df1.columns)
print(df2.columns)

Index(['Body', 'Label'], dtype='object')
Index(['Body', 'Label'], dtype='object')


In [188]:
df_final = pd.concat([df1, df2])
df_final = df_final.rename(columns={"Label": "Spam"})
df_final.sample(n=5).head() 


,Body,Spam
2295,Subject: you don _ t know how to get into search engine results ?\n submitting your website in search engines may increase\n your online sales dramatically .\n lf you invested time and money into ...,1
2393,"On Sun, 8 Sep 2002, Gordon Mohr wrote:> OK, then. Consider a population of 1,000,000. 500,000 men each\n> pair off with 500,000 women. Then, 1 man, let's call him ""Wilt"",\n> also has sex with the ...",0
4409,"Hi all, I posted the below question to this list so I thought I'd update you\nall. Thanks for your suggestions.Well, I'd decided to bite the bullet and spend big on a new case and CPU\nfan.I bough...",0
1585,"Subject: logo , stationer , website design and so much more !\n lt is really hard to recollect a company : the\n market is full of suqgestions and the information isoverwhelminq ; but a good\n cat...",1
5961,"You are receiving this email because you signed up to \nreceive one of our free reports. If you would prefer \nnot to receive messages of this type, please \nunsubscribe by following the instructi...",0


## Preprocesamiento

Aplique las técnicas de pre – procesamiento de lenguaje natural que considere necesarias 
(conversión de minúsculas, mayúsculas, eliminación de acentos, expansión de contracciones, eliminación de stop
words, etc.)

In [189]:
#removemos espacios al inicio y final
df_final['Body'] = df_final['Body'].str.strip()

In [190]:
#conversion a minusculas
df_final['Body'] = df_final['Body'].str.lower()


In [191]:
#remover acentos
df_final['Body'] = df_final['Body'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [192]:
#expandir contracciones
import contractions
df_final["Body"] = df_final["Body"].apply(lambda x: contractions.fix(str(x)))

In [193]:
#eliminar caracteres especiales, numeros y simbolos
import re
def removerCaracteresEspecialesNumerosSimbolos(texto, removerDigitos =False):
    patron = r'[^a-zA-Z0-9\s]' if not removerDigitos else r'[^a-zA-Z\s]'
    texto = re.sub(patron,'', texto)
    return texto
oracion = 'Hoy es el partido de fútbol a las 7:45 @$%&'

df_final["Body"] = df_final["Body"].apply(lambda x: removerCaracteresEspecialesNumerosSimbolos(x, True))


In [194]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joopa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joopa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [195]:
#eliminar stop words
stop_words = nltk.corpus.stopwords.words('english')
df_final["Body"]= df_final["Body"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop_words))


In [196]:
df_final.sample(n=5).head() 


,Body,Spam
2829,stateful would seem worth effort although guess might help ddos attackssounds snakeoilish biased lots experience firewalls proxy servers limitations thereofken original message rohit khare mailtok...,0
2027,subject story stetson pleasant destroy weller melanie lame northern bulwark evocation chromatin n l n e n v e r p l e g r e e obtain diploma degree master send certificate countries worldwide cons...,1
582,subject free glucose meter enroll today see qualify free glucose meter ship diabetic testing supplies direct door free home delivery bill insurance medicare visit http www diabetesecare com veloznet,1
3608,also identified spam everything inbox mime structure html parts several messages saved zope geeks list happened using mime andor html know strangest implied claim hates mime independent html examp...,0
4516,actually though case sensitive seems trouble wtih papchap authentication well might scripton friday july liam bedford wrote fri jul hamiltondavid hpirelandex claiming think hi trying find oceanfre...,0


## Representación de texto


Utilice los modelos de BoG (para n = 1,2) y TF-IDF. Muestre algunos ejemplos de los mensajes en su
representación numérica.

### Modelo Bag of Words

In [197]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.2, max_df=0.8)
# Min_df y Max_df nos sirven para controlar el porcentaje mínimo y máximo de apariciones de un token
cv_matrix = cv.fit_transform(df_final["Body"])
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[3, 1, 1, ..., 0, 0, 0],
       [0, 1, 2, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [3, 0, 0, ..., 1, 0, 1]], dtype=int64)

In [200]:
# Palabras únicas del corpus
vocabulario = cv.get_feature_names()
# Mostrar el vector
df_cv_BagOfWords = pd.DataFrame(cv_matrix, columns=vocabulario)

C:\Users\joopa\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [201]:
df_cv_BagOfWords.head()

,email,get,list,new,one,please,subject,time,would
0,3,1,1,1,0,2,0,0,0
1,0,1,2,0,0,1,0,0,0
2,0,1,1,0,0,1,0,0,0
3,1,2,0,1,3,1,0,0,0
4,0,1,2,0,0,1,0,0,0


### Modelo Bag of N-Grams

In [211]:
bv = CountVectorizer(ngram_range=(1,2), min_df=0.1, max_df=0.9)
bv_matrix = bv.fit_transform(df_final2["Body"])

bv_matrix = bv_matrix.toarray()
vocabulario = bv.get_feature_names()
df_cv_BagOfNGrams = pd.DataFrame(bv_matrix, columns=vocabulario)

C:\Users\joopa\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [212]:
df_cv_BagOfNGrams.head()

,10,2000,2002,also,best,business,cc,click,com,company,...,thanks,time,today,us,use,want,way,well,work,would
0,0,0,0,0,3,0,0,2,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,3,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


### Modelo TF -IDF

In [221]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0.1, max_df=0.9, use_idf=True)
tv_matrix = tv.fit_transform(df_final2["Body"])
tv_matrix = tv_matrix.toarray()

vocabulario = tv.get_feature_names()
df_cv_TFIDF = pd.DataFrame(np.round(tv_matrix, 2), columns=vocabulario)

C:\Users\joopa\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [222]:
df_cv_TFIDF.head()

,10,2000,2002,also,best,business,cc,click,com,company,...,thanks,time,today,us,use,want,way,well,work,would
0,0.0,0.0,0.0,0.0,0.50,0.0,0.0,0.34,0.0,0.0,...,0.0,0.0,0.0,0.15,0.00,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.32,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.43,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.28,0.0,0.0,0.09,0.0,0.0,...,0.0,0.0,0.0,0.08,0.00,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.33,0.0,0.0,0.0,0.0,0.0


## Parte 2 – Implementación del modelo 
## Separación de datos 
- Datos de entrenamiento: 70% 
- Datos de prueba: 30% 


## Implementación 
Utilice el algoritmo multinomial de NaiveBayes para entrenar el modelo con cada uno de los modelos 
de representación numérico. Muestre los valores obtenidos para las siguientes métricas: 

- Matriz de confusión 
- Precision 
- Recall 
- F1 Score 

## Conclusion

Compare los valores para cada modelo de representación numérico. ¿Qué modelo produjo el mejor 
resultado, BoG o TF-IDF? ¿A que se debe la mejora? Explique los valores obtenidos en las métricas 
para el mejor modelo. 